# Snowboard deals

This LLM is going to summarize the current deals on snowboard top pages.

In [73]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL_GPT = 'gpt-4o-mini'
gpt_openai = OpenAI()

MODEL_LLAMA = 'llama3.2'
ollama_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [75]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class BlueTomatoWebsite:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    base_url = 'https://www.blue-tomato.com'
    links = []

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content

        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # get all links inside <main> tag
        main_tag = soup.find('main')
        links = [link.get('href') for link in main_tag.find_all('a')]
        # Only include links with product and snowboard word in the url, and append the base url to the link
        links = [self.base_url + link for link in links if link and '/product/' in link]
        self.links = links[:20]

        if soup.body and soup.body.find('section', {'data-testid': 'DESCRIPTION'}):
            self.descripcion = soup.body.find('section', {'data-testid': 'DESCRIPTION'}).get_text(separator="\n", strip=True)
        else:
            self.descripcion = ""

        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
bluetomato_deals = BlueTomatoWebsite('https://www.blue-tomato.com/es-ES/products/categories/Tienda+de+snowboard-00000000--Equipacion+de+snowboard-000099C0--Tablas+de+snowboard-00000001/?sort=-bestseller')

bluetomato_deals.links

In [77]:
# perform a request to each link and get the html

snowboard_deals = []
for link in bluetomato_deals.links:
    snowboard_deals.append(BlueTomatoWebsite(link))



In [86]:
# get the description of the all snowboard deals add the url to the description
snowboard_descriptions = [f"URL: {deal.url}\n{deal.descripcion}\n\n" for deal in snowboard_deals]

snowboard_expert_system_prompt = """
You are a snowboard expert. You are given a description of a list of snowboards and you need to return a ordered list of the best snowboards for a specific style and level.
Respond in table using markdown format, with the snowboard name and the url, and the reason why it is the best snowboard for the style and level.
"""

question = "What is the best top 5 snowboard on this list by style and level?"
question += "\n\n".join(snowboard_descriptions)

In [ ]:
print(snowboard_expert_system_prompt)
print(question)

In [87]:
def stream_response_to_markdown(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

def ask_gpt_expert(prompt):
    stream = gpt_openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": snowboard_expert_system_prompt}, {"role": "user", "content": question}
        ],
        stream=True
    )
    return stream_response_to_markdown(stream)

In [88]:
ask_gpt_expert(question)

Aquí tienes un resumen de los cinco mejores snowboards según el estilo y el nivel de freestyle:

| Snowboard Name                       | URL                                                                                                                                                                                                                                          | Reason                                                                                                       |
|--------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| **Forum Production 003**             | [Forum Production 003](https://www.blue-tomato.com/es-ES/product/Forum-Park+003+2024+Snowboard-715031/)                                                                                                                                 | Diseñada específicamente para el park, con un camber suave, True Twin y un flex ideal para freestyle.        |
| **Lib Tech Legitimizer**             | [Lib Tech Legitimizer](https://www.blue-tomato.com/es-ES/product/Lib+Tech-Legitimizer+2025+Snowboard-729921/)                                                                                                                           | Perfecta para el park, con excelente pop y control, ideal para riders intermedios y avanzados.                |
| **Burton Process**                   | [Burton Process](https://www.blue-tomato.com/es-ES/product/Burton-Process+2025+Snowboard-729450/)                                                                                                                                        | Ofrece un camber PurePop juguetón, ideal para freestyle all-mountain, adecuado para riders avanzados.       |
| **Bataleon Evil Twin+**              | [Bataleon Evil Twin+](https://www.blue-tomato.com/es-ES/product/Bataleon-Evil+Twin+2025+Snowboard-734299/)                                                                                                                               | Diseñada para freestyle avanzado, con una construcción ligera para un alto rendimiento en el park.            |
| **CAPiTA Defenders Of Awesome**      | [CAPiTA Defenders Of Awesome](https://www.blue-tomato.com/es-ES/product/CAPiTA-Defenders+Of+Awesome+2025+Snowboard-732362/)                                                                                                            | Reconocida como una de las mejores tablas de freestyle all-mountain, combina permisión y agarre en todo terreno.|

Esta selección está basada en las características específicas para el freestyle y la habilidad de los riders avanzados.

'Aquí tienes un resumen de los cinco mejores snowboards según el estilo y el nivel de freestyle:\n\n| Snowboard Name                       | URL                                                                                                                                                                                                                                          | Reason                                                                                                       |\n|--------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|\n| **Forum Production 003**             | [Forum Production 003](https://www.blue-tomato.com/es-ES/product/Forum-Park

In [90]:
# Get Llama 3.2 to answer

def ask_llama_expert(question):
    stream = ollama_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[{"role": "system", "content": snowboard_expert_system_prompt}, {"role": "user", "content": question}],
        stream=True
    )
    return stream_response_to_markdown(stream)



In [92]:
ask_llama_expert(question)

Aquí te presento un resumen de las características y destros de cada tabla de snowboard:

**1. La Defenders of Awesome de Capita**

* Estilo: All-mountain con experiencia
* Forma: True Twin
* Perfil: Resort V1 (camber tradicional entre las fijaciones con sutil rocker en nose y tail)
* TPU: Flex medio, ideal para freestyle y all-mountain
* Construcción:
 + Núcleo P2 Superlight Core con certificación FSC
 + Protección lateral de Kevlar Capita Fortress
 + Suela Quantum Drive con resina Magic Bean vegetal
 * Características clave:
 - Perfil híbrido Resort V1
 - Protección lateral de kevlar Fortress
 - Suela Quantum Drive con resina Magic Bean

**2. La Defenders of Awesome de Capita (edición 2025)**

* Estilo: All-mountain con experiencia
* Forma: True Twin
* Perfil: Resort V1
* TPU: Flex medio, ideal para freestyle y all-mountain
* Construcción:
 + Núcleo P2 Superlight Core con certificación FSC
 + Protección lateral de Kevlar Capita Fortress
 + Suela Quantum Drive con resina Magic Bean vegetal
 * Características clave:
 - Perfil híbrido Resort V1
 - Protección lateral de kevlar Fortress
 - Suela Quantum Drive con resina Magic Bean

**3. La Defenders of Awesome en la Tienda**

 Aquella no era tan buena como la del capi y tuvo que ser eliminada.

**4. La CAPiTA Defenders Of Awesome 2025**

* Estilo: All-mountain
* Forma: Twin
* Perfil: Resort V1
* TPU: Flex medio, ideal para freestyle y all-mountain
* Construcción:
 + Núcleo P2 Superlight Core con certificación FSC
 + Protección lateral de Kevlar Fortress
 + Suela Quantum Drive con resina Magic Bean vegetal
 * Características clave:
 - Perfil híbrido Resort V1

**5. La Defenders of Awesome**

* Estilo: All-mountain
* Forma: Twin
* Perfil: Resort V1
* TPU: Flex medio, ideal para freestyle y all-mountain
* Construcción:
 + Núcleo P2 Superlight Core con certificación FSC
 + Protección lateral de Kevlar Fortress
 * Características clave:
 - Perfil híbrido Resort V1

**6. La Defenders of Awesome**

* Estilo: All-mountain
* Forma: Twin
* Perfil: 
 + TPU: No especificado.
  * Características clave:
   - Protección lateral de kevlar Fortress
    - Suela Quantum Drive con resina Magic Bean

'Aquí te presento un resumen de las características y destros de cada tabla de snowboard:\n\n**1. La Defenders of Awesome de Capita**\n\n* Estilo: All-mountain con experiencia\n* Forma: True Twin\n* Perfil: Resort V1 (camber tradicional entre las fijaciones con sutil rocker en nose y tail)\n* TPU: Flex medio, ideal para freestyle y all-mountain\n* Construcción:\n + Núcleo P2 Superlight Core con certificación FSC\n + Protección lateral de Kevlar Capita Fortress\n + Suela Quantum Drive con resina Magic Bean vegetal\n * Características clave:\n - Perfil híbrido Resort V1\n - Protección lateral de kevlar Fortress\n - Suela Quantum Drive con resina Magic Bean\n\n**2. La Defenders of Awesome de Capita (edición 2025)**\n\n* Estilo: All-mountain con experiencia\n* Forma: True Twin\n* Perfil: Resort V1\n* TPU: Flex medio, ideal para freestyle y all-mountain\n* Construcción:\n + Núcleo P2 Superlight Core con certificación FSC\n + Protección lateral de Kevlar Capita Fortress\n + Suela Quantum Dri